# CoCo: Learning how to use Fatiando a Terra tools

_by [Santiago Soler](https://github.com/santisoler/), [Agustina Pesce](https://github.com/aguspesce/) and [Andrea Balza](https://github.com/andieie)_

In this notebook we will learn how we can use the Fatiando a Terra tools to
process gravity data (or any spatial data).

Processing gravity data might seem easy, although there are a lot of details
that we must to take into account.
Nevertheless, the idea of this notebook is not to become experts on gravity
modelling, but to use it as an excuse for learning more Python and how should
we approach learning a new library.

A very nice place to start learning a new tool is its website and its
documentation pages.

The Fatiando a Terra project has its own website: https://www.fatiando.org

The project is composed by different open-source Python libraries, each one
with a specific goal and set of problems that they tackle.
Each one of these libraries has its own documentation pages:
- Pooch: https://www.fatiando.org/pooch
- Boule: https://www.fatiando.org/boule
- Verde: https://www.fatiando.org/verde
- Harmonica: https://www.fatiando.org/harmonica

## Goals

1. Fetch and cache the data using [Pooch](https://github.com/fatiando/pooch)
1. Calculate the gravity disturbance using [Boule](https://github.com/fatiando/boule)
1. Interpolate data and apply map projections to grids using [Verde](https://github.com/fatiando/verde)
1. Perform topographic correction and equivalent-source interpolation using [Harmonica](https://github.com/fatiando/harmonica)
---
En este notebook vamos a aprender cómo podemos utilizar las herramientas de Fatiando a Terra para procesar datos de gravedad (o cualquier tipo de dato espacial).

El procesamiento de datos gravimétricos puede parecer sencillo, sin embargo hay muchos detalles que se deben tener en cuenta. Sin embargo, la idea de este notebook no es convertirnos en expertxs en el modelado gravimétrico, sino utilizarlo como excusa para aprender más sobre Python y cómo debemos proceder a la hora de aprender una nueva librería.

Un muy buen lugar para comenzar a aprender una nueva librería es su sitio web y sus páginas de documentación.

El proyecto Fatiando a Terra posee su propia página web: https://www.fatiando.org

El proyecto está compuesto por diferentes librerías open-source escritas en Python, cada una con objetivos específicos y problemas que permiten resolver. Cada una de estas librerías posee sus propias páginas de documentación:

- Pooch: https://www.fatiando.org/pooch
- Boule: https://www.fatiando.org/boule
- Verde: https://www.fatiando.org/verde
- Harmonica: https://www.fatiando.org/harmonica
    
## Objetivos

1. Importar librerías nuevas
1. Leer documentación (online y docstrings)
1. Descargar datos con Pooch
1. Manipular datos espaciales con Verde
1. Calcular gravedad normal con Boule
1. Procesar datos gravimétricos con Harmonica


## Study area 

The **Bushveld Igneous Complex** is located in South Africa and is the largest known layered igneous intrusion. It has been tilted and eroded forming the outcrops around what appears to be the edge of a great geological basin: the Transvaal Basin. It is approximately 2 billion years old and is divided into four different limbs: the northern, southern, eastern, and western limbs. The Bushveld Complex comprises the Rustenburg Layered suite, the Lebowa Granites and the Rooiberg Felsics, that are overlain by the Karoo sediments. See [Webb et al. (2004)](https://doi.org/10.2113/107.1-2.207) for an overview and previous interpretations of the Bushveld in depth.

<figure>
<img src="images/bushveld_igneous_complex_geology.jpg" alt="Geologic map of the Bushveld Complex">
<figcaption><em>
    Geology and mineral exploration sites of the Bushveld Igneous Complex 
    (<a href="https://en.wikipedia.org/wiki/File:Bushveld_Igneous_Complex.png">public domain</a>).
</em></figcaption>
</figure>

## Import packages | Importar paquetes

Import Numpy, Pandas and Matplotlib for managing and plotting our data.

Importemos Numpy, Pandas y Matplotlib para gestionar nuestros datos y graficarlos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Lets import **Xarray**, a library similar to Pandas, but it allows us to work with multidimensional data besides tabular data.

Importemos **Xarray**, una librería similar a Pandas, pero que nos permite trabajar con datos multidimensionales en vez de datos tabulados.

In [ ]:
import xarray as xr

Lets import **pyproj**: it will help us to convert geographic coordinates into Cartesian coordinates through map projections.

Importemos **pyproj** para poder convertir coordeanadas geográficas en coordenadas Cartesianas mediante proyecciones.



In [ ]:
import pyproj

Finally, lets import some Fatiando libraries.

Finalmente, importemos algunas librerías de Fatiando.

In [ ]:
import pooch
import verde as vd
import boule as bl
import harmonica as hm

## Fetch data | Descarguemos datos

In this notebook we will use three different types of data:

- **gravity data**: we will use a set of gravity data over South Africa. These data consist of measurements of the gravity acceleration produced by Earth at different points of observation located on South Africas surface topography. Gravity data are in mGal, a unit which is equivalent to 1e5 m/s2. 

- **topography of South Africa** : the last step of this notebook aims to remove the gravity effect of the topography from our data. In order to do this, we need the DEM (digital elevation model) or a topography grid.

These data are available with a Creative Common Attribution license in https://doi.org/10.5281/zenodo.5167357.
Let's see how we can use *Pooch* to download this data to our computer to then load it to our notebook. 

Lets define the Digital Object Identified (DOI) of the Zenodo file where our data are located.

---
En este notebook vamos a utilizar tres tipos de datos diferentes:

- **datos gravimétricos**: vamos a utilizar un conjunto de datos gravimétricos sobre Sudáfrica.Estos datos consisten en mediciones de la aceleración gravitatoria producida por el planeta Tierra sobre diferentes puntos de observación ubicados sobre la superficie de Sudáfrica. Los datos gravimétricos vienen dados en mGal, una unidad que equivale a 1e5 m/s^2.
- **topografía de Sudáfrica**: el último paso de este notebook consistirá en remover el efecto gravitatorio de la topografía de Sudáfrica de nuestros datos. Para ello necesitamos un DEM (modelo de elevación digital) o una grilla de topografía.

Todos estos datos se encuentran disponibles bajo licencia Creative Commons Attribution en https://doi.org/10.5281/zenodo.5167357. Veamos cómo podemos utilizar Pooch para descargar estos datos a nuestra computadora para luego poder cargarlos en nuestros notebooks.

Definimos el Digital Object Identified (DOI) del archivo de Zenodo en el cual se encuentran nuestros datos.

In [ ]:
doi = "10.5281/zenodo.5167357"

Let's use the function `pooch.retrieve` to download the data file `southern-africa-gravity.csv.xz` and `earth-topography-10arcmin.nc` . Also, we will use the checksum so Pooch could verify if these files are corrupted or not. 

Utilicemos la función `pooch.retrieve` para descargar los archivos de datos `southern-africa-gravity.csv.xz` y `earth-topography-10arcmin.nc`. Además, vamos a utilizar la suma de comprobación de estos archivos, de forma tal que Pooch pueda verificar que los archivos que se descarguen no estén corruptos.

In [ ]:
gravity = f"doi:{doi}/southern-africa-gravity.csv.xz"
gravity_hash = "md5:1dee324a14e647855366d6eb01a1ef35"

path_gravity = pooch.retrieve(gravity, known_hash=gravity_hash, progressbar=True)
path_gravity

In [ ]:
topography = f"doi:{doi}/earth-topography-10arcmin.nc"
topography_hash = "md5:c43b61322e03669c4313ba3d9a58028d"

path_topography = pooch.retrieve(
    topography, known_hash=topography_hash, progressbar=True
)
path_topography

## Load the data | Cargar los datos

Let's start with the **gravity data**. Because the gravity data that we downloaded is a CSV file (zipped in an .xz), we will use Pandas to read them.

Comencemos por los **datos gravimétricos**. Dado que los datos de gravedad que descargamos consiste en un archivo CSV (comprimido en un .xz), vamos a utilizar Pandas para leerlo.

In [ ]:
data = pd.read_csv(path_gravity)
data

The topography data are in regular grids as netCDF (.nc) files. To load these data we will use Xarray.

Los datos de **topografía** vienen dados en grillas regulares en archivos netCDF (.nc). Para cargar estos datos vamos a hacer uso de Xarray.

In [ ]:
topography = xr.load_dataarray(path_topography)
topography

## Plot the data | Grafiquemos los datos

All of the data that we downloaded are georeferenced in eliptical coordinates (lat/long). We will use matplotlib to visualize the data that we just downloaded.

> **Note**:
> To plot georeferenced data, we recommend libraries that are more suitable, like **pyGMT**
> In order to simpligy this notebook, we will use matplotlib

---
Todos los datos que descargamos se encuentran georeferenciados en coordenadas elipsoidales (latitud, longitud). 
Utilicemos matplotlib para poder visualizar los datos que descargamos.

> **Nota**
> Para graficar datos georeferenciados es recomendable utilizar librerías más adecuadas, como **pyGMT**.
> Con el objetivo de mantener este notebook más simple, utilizaremos matplotlib.

In [ ]:
fig = plt.figure(figsize=(12, 12))
plt.scatter(data.longitude, data.latitude, c=data.gravity_mgal, s=1)
plt.gca().set_aspect("equal")
plt.show()

## Crop the data to the Bushveld | Recortemos los datos sobre Bushveld

As we can see, the gravity data are of all of South Africa. However, our region of interest is the Bushveld complex. We will zoom in to our region and add more external data, so we also avoid boundary effects of previous processes. 

Lets define the Bushveld region in degrees ( west, east, south, north):

Como podemos ver, los datos de gravedad pertenecen a toda Sudáfrica, sin embargo nuestra zona de interés es la región de Bushveld. Las grillas de geoide y topografía vamos a recortarlas a regiones un poco más extensas, con el objetivo de evitar efectos de borde en los posteriores procesos.

Definamos la región de Bushveld en grados (oeste, este, sur, norte):

In [ ]:
region = (25, 32, -27, -23)

We define the region and then add padding cells to the grid of interest. We will use `vd.pad_region()` function from Verde:

Agrandemos la region para utilizar con las grillas. Para ello haremos uso de la función `vd.pad_region()` de Verde:

In [ ]:
region_pad = vd.pad_region(region, pad=5)
region_pad

We can use `vd.inside` function in Verde to obtain the data that are inside of our region and then cut them using Pandas.

Podemos utilizar la función `vd.inside` de Verde para obtener qué datos se encuentran dentro de nuestra región y luego recortarlos con Pandas.

In [ ]:
inside_points = vd.inside((data.longitude, data.latitude), region)

data = data[inside_points]
data

To cut the grid directly we could use the `sel()` method in the `xarray.DataArray`. 

Para recortar grillas podemos utilizar directamente el método `sel()` de los `xarray.DataArray`.

In [ ]:
topography = topography.sel(
    longitude=slice(region_pad[0], region_pad[1]),
    latitude=slice(region_pad[2], region_pad[3])
)
topography

## Plot the data

In [ ]:
plt.figure(figsize=(12, 12))
plt.scatter(data.longitude, data.latitude, c=data.gravity_mgal, s=10)
plt.colorbar(label="mGal", shrink=0.6)
plt.gca().set_aspect("equal")
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
topography.plot()
plt.scatter(data.longitude, data.latitude, c="black", s=0.2)
plt.gca().set_aspect("equal")
plt.show()

## Project the data and the grid | Proyectando los datos y la grilla


Because our region of interest is small enough, we can use a projection to transform our data from geographic coordinates to cartesian coordinates. This simplifies our tasks like interpolating and topographic corrections, that tend to be alot faster in castesian coordinates than espheric of geographic coordinates.

We will make use of `pyproj` to create a Mercator projection type with latitude in "real scale" centered on the data.

Dado que nuestra región es los suficientemente pequeña podemos utilizar una proyección para transformar nuestros datos en coordenadas geográficas (longitud, latitud) a coordenadas Cartesianas. Esto simplifica mucho tareas como  la interpolación y las correcciones topográficas, que suelen ser mucho más rápidas en coordenadas Cartesianas que en coordenadas esféricas o geográficas.

Haremos uso de `pyproj` para crear una proyección de tipo Mercator con la latitud de la "escala real" centrada en los datos.


In [ ]:
projection = pyproj.Proj(proj="merc", lat_ts=data.latitude.mean())

Let's use this `projection` to transform the gravity data coordinates and then include them in the dame `pandas.DataFrame`
Utilicemos esta `projection` para transformar las coordenadas de nuestros datos de gravedad y luego incluirlas en el mismo `pandas.DataFrame`.

In [ ]:
easting, northing = projection(data.longitude.values, data.latitude.values)
data = data.assign(easting_m=easting, northing_m=northing)
data

Projecting a grid is more complicated, it involves interpolating with the goal of having equidistance nodes. Verde has a function called `project_grid` which calculates this automatically. We will use a nearest neighbor interpolation because it is one the faster methods and the projections that we choose will not deform our data significantly. 

Projectar grillas es más complicado, ya que involucra realizar algunas interpolaciones a fines de mantener los nodos equiespaciados. Verde posee la función `project_grid` que nos permite calcular todo esto de forma automática. Utilizaremos una interpolación a primeros vecinos dado que es una de las más rápidas y la proyección que elegimos no deforma tanto nuestros datos.

In [ ]:
topography_proj = vd.project_grid(topography, projection, method="nearest")
topography_proj

## Gravity disturbance | Disturbio de gravedad

The gravity disturbance is defined as:

$$ \delta g(\lambda, \phi, h) = g(\lambda, \phi, h) - \gamma(\lambda, \phi, h) $$

where $(\lambda, \phi, h)$ are the longitudinal, latitudinal and altitude coordinates of the observation point, and $\gamma$ is the **normal gravity** generated by the reference ellipsoid. Note that these three quantities are located at the same observation point.

> **Note**
> The altitudes of the data we download are referred to sea level, that is, to the geoid. However, it is recommended to work with altitudes above the reference ellipsoid.
> In order to keep this notebook short, we are going to skip the step of transforming altitudes on geoid to altitudes on ellipsoid.
> Since the differences between the two are small, we won't see any noticeable effects of skipping that step.
> For more information on how to perform this step, refer to: https://github.com/fatiando/tutorials

The values ​​of $g$ on the observation points are given in the data that we downloaded, therefore we have to calculate the normal gravity in the same points.
Boule allows us to define a reference ellipsoid (we will use WGS84) and calculate its gravitational effect at any external point.

---

El disturbio de gravedad se define como:

$$ \delta g(\lambda, \phi, h) = g(\lambda, \phi, h) - \gamma(\lambda, \phi, h) $$

donde $(\lambda, \phi, h)$ son las coordenadas longitudinales, latitudinales y la altitud del punto de observación, y $\gamma$ es la **gravedad normal** generada por el elipsoide de referencia. Nótese que estas tres cantidades se ubican en el mismo punto de observación.

> **Nota**
> Las altitudes de los datos que descargamos están referidas al nivel del mar, es decir, al geoide. Sin embargo, es recomendable trabajar con altitudes sobre el elipsoide de referencia.
> Con el objetivo de mantener este notebook breve, vamos a saltarnos el paso de transformar las altitudes sobre geoide a altitudes sobre elipsoide.
> Debido a que las diferencias entre ambas son pequeñas, no veremos efectos apreciables al omitir ese paso.
> Para más información sobre cómo se puede realizar este paso referirse a: https://github.com/fatiando/tutorials

Los valores de $g$ sobre los puntos de observación vienen dados en los datos que descargamos, por lo tanto nos resta calcular la gravedad normal en los mismos puntos.
Boule nos permite definir un elipsoide de referencia (usaremos el WGS84) y calcular su efecto gravitatorio en cualquier punto exterior.

Let's define the reference elipsoid

Definimos el elipsoide de referencia

In [ ]:
ellipsoid = bl.WGS84
ellipsoid

We will now calculate normal gravity over our observation points

Calculamos gravedad normal sobre los puntos de observación

In [ ]:
normal_gravity = ellipsoid.normal_gravity(data.latitude, data.height_sea_level_m)
normal_gravity

Calculamos el disturbio de gravedad y lo agregamos a nuestro `DataFrame`

In [ ]:
gravity_disturbance = data.gravity_mgal - normal_gravity

data = data.assign(gravity_disturbance_mgal=gravity_disturbance)
data

Lets plot the gravity disturbance. Since we will make many plots throughout this notebook, it is better to create a function for this purpose:

Grafiquemos el disturbio de gravedad. Como vamos a realizar múltiples gráficas a lo largo de este notebook, es mejor crear una funcion para ello:

In [ ]:
def scatter_plot_gravity(easting, northing, gravity_data, title=None):
    """
    Generate scatter plots of gravity data
    
    The colorbar will be centered around cero and using the seismic colormap.
    
    Parameters
    ----------
    easting: array
        Easting coordinates of the gravity data in meters.
    northing : array
        Northing coordinates of the gravity data in meters.
    gravity_data : array
        Values of the gravity data in mgal.
    title : str (optional)
        Title of the plot.
    """
    maxabs = vd.maxabs(gravity_data)
    plt.figure(figsize=(12, 12))
    plt.scatter(
        easting,
        northing,
        c=gravity_data,
        s=10,
        cmap="seismic",
        vmin=-maxabs,
        vmax=maxabs,
    )
    plt.xlabel("easting [m]")
    plt.ylabel("northing [m]")
    if title is not None:
        plt.title(title)
    plt.colorbar(label="mGal", shrink=0.6)
    plt.gca().set_aspect("equal")

Now we can plot the gravity disturbance using the function we wrote.

Ahora sí, grafiquemos el disturbio de gravedad usando la función que escribimos.

In [ ]:
scatter_plot_gravity(
    data.easting_m,
    data.northing_m,
    data.gravity_disturbance_mgal,
    title="Gravity disturbance",
)

## Topographic correction | Corrección topográfica

The gravity disturbance contains all the information about the gravitational acceleration generated by anomalous masses. One of the things that contributes the most to the gravity disturbance is the topography: those bodies that are above the reference ellipsoid.
The effect of the topography hides the effect of buried bodies that are of greater interest to us (in this case the igneous rocks of Bushveld).
Let us then remove the effect of the topography of the gravity disturbance.

Harmonica allows us to calculate the gravitational effects of topography by modeling it with simple geometries such as rectangular prisms.

The `prism_layer` function creates a prism model from a topographical surface that undulates around a reference level. In our case the reference level will be the ellipsoid (zero altitude) and the surface will be the topography.

The hardest part is assigning the density to each of these prisms:

* $\rho = 2670\ kg/m^3$ for continental regions with positive topography (above the ellipsoid)
* $\rho = 1040 - 2670\ kg/m^3$ in the oceans (1040 is the approximate density of sea water)


<figure>
<img src="images/topographic-correction.svg" alt="Sketch of the surfaces and masses involved in topographic correction of gravity disturbances.">
<figcaption><em>
    Sketch of the surfaces and masses involved in topographic correction of gravity disturbances.
    (CC BY).
</em></figcaption>
</figure>

> **Note**
> There are some details to consider when making this type of topographic corrections: the sea level does not coincide with the surface of the ellipsoid, and therefore there are bodies of water above and below the ellipsoid.
> In the interest of keeping this notebook brief, we have left out these details.
> For more information on this, you can refer to: https://github.com/fatiando/tutorials

--- 

El disturbio de gravedad contiene toda la información acerca de la aceleración gravitatoria generada por masas anómalas. Uno de los conjuntos que mayor contribuye al disturbio de gravedad es la topografía: aquellos cuerpos que se encuentran por encima del elipsoide de referencia.
El efecto de la topografía nos esconde el efecto de cuerpos enterrados que nos son de mayor de interés (en este caso las rocas ígneas de Bushveld).
Quitemos entonces el efecto de la topografía del disturbio de gravedad.

Harmonica nos permite calcular los efectos gravitatorios de la topografía modelándola con geometrías sencillas como son los prismas rectangulares.

La función `prism_layer` crea un modelo de prismas a partir de una superficie topográfica que ondula alrededor de un nivel de referencia. En nuestro caso el nivel de referencia será el elipsoide (altitud cero) y la superficie será la topografía.

La parte más difícil es asignar la densidad a cada uno de estos prismas:

* $\rho = 2670\ kg/m^3$ para regiones continentales con topografía positiva (por encima del elipsoide)
* $\rho = 1040 - 2670\ kg/m^3$ en los oceanos (1040 es la densidad aproximada del agua del mar)

> **Nota**
> Existen algunos detalles a considerar al realizar este tipo de correcciones topográficas: el nivel del mar no coincide con la superficie del elipsoide, y por ende hay masas de agua por encima y por debajo del elipsoide.
> Con el objetivo de mantener este notebook breve, hemos obviado estos detalles.
> Para más información al respecto puede referirse a: https://github.com/fatiando/tutorials

In [ ]:
density = np.where(topography_proj > 0, 2670, 1040 - 2670)

topography_model = hm.prism_layer(
    coordinates=(topography_proj.easting, topography_proj.northing),
    surface=topography_proj,
    reference=0,
    properties={"density": density},
)
topography_model

We now calculate the gravity effect of these prisms over our observation points

Calculemos ahora el efecto gravitatorio de estos prismas sobre los puntos de observación

In [ ]:
coordinates = (data.easting_m, data.northing_m, data.height_sea_level_m)

terrain_effect = topography_model.prism_layer.gravity(coordinates, field="g_z")
terrain_effect

We store these values in our `DataFrame` 

Guardemos estos valores en nuestro `DataFrame`

In [ ]:
data = data.assign(terrain_effect_mgal=terrain_effect)
data

We calculate our **Bouguer gravity disturbance** that is defined as the gravity disturbance without the effect of topography. 

Y calculemos el **disturbio de Bouguer** que se define como el disturbio de gravedad menos el efecto de la topografía

In [ ]:
gravity_bouguer = data.gravity_disturbance_mgal - data.terrain_effect_mgal

data = data.assign(gravity_bouguer_mgal=gravity_bouguer)
data

In [ ]:
scatter_plot_gravity(
    data.easting_m,
    data.northing_m,
    data.gravity_bouguer_mgal,
    title="Bouguer gravity disturbance",
)

## Regional-residual separation

The Bouguer gravity disturbance is mainly dominated by the effects of very deep masses located at the interface between the crust and the mantle (Mohorovicic discontinuity).
If we want to observe the signal from the Bushveld igneous bodies (called **residuals**) we need to remove the effect of these deep masses (called **regional**).

For this we are going to use the technique called **equivalent source**:
- we will define a set of deep point sources
- we will adjust their masses in such a way that they generate a field approximately equal to that of the Bouguer disturbance
- we will calculate the **regional** effect as the effect generated by these sources in the observation points

<figure>
<img src="images/equivalent-sources.svg" alt="Sketch equivalent sources method to perform interpolation or predict residuals.">
<figcaption><em>
    Sketch equivalent sources method to perform interpolation or predict residuals.
    (CC BY).
</em></figcaption>
</figure>

For this we will make use of Harmonica's `EquivalentSources`, which allow us to carry out all these steps through a simple interface to use.

--- 
El disturbio de gravedad de Bouguer está dominado principalmente por los effectos de masas muy profundas ubicadas en la interfaz entre la corteza y el manto (discontinuidad de Mohorovicic).
Si queremos observar la señal de los cuerpos ígneos de Bushveld (llamados **residuales**) es necesario que quitemos el efecto de estas masas profundas (llamdas **regionales**).

Para ello vamos a utilizar la técnica de **fuentes equivalentes**:
- definiremos un conjunto de fuentes puntuales profundas
- ajustaremos sus masas de forma tal que generen un campo aproximadamente igual al del disturbio de Bouguer
- calcularemos el efecto **regional** como el efecto que generan estas fuentes en los puntos de observación

Para ello haremos uso de las `EquivalentSources` de Harmonica, que nos permiten llevar a cabo todos estos pasos mediante una interfaz simple de utilizar.

We define the equivalent sources located at 500 km depth ( the damping parameter lets you smooth the masses of the sources. A larger value generates smoother solutions)

Definimos las fuentes equivalentes ubicadas a 500km de profundidad (el damping es un parámetro de amortiguamiento que permite "suavizar" las masas de las fuentes, un valor alto genera soluciones más suaves).

In [ ]:
deep_sources = hm.EquivalentSources(damping=1000, depth=500e3)

We fit the deep sources using the Bouguer gravity disturbance values. 

Ajustamos las masas de las fuentes con los datos del disturbio de Bouguer

In [ ]:
coordinates = (data.easting_m, data.northing_m, data.height_sea_level_m)

deep_sources.fit(coordinates, data.gravity_bouguer_mgal)

Now we calculate the **regional** effect of the gravity of these deep sources

Calculamos el efecto **regional** como el efecto gravitatorio de estas fuentes profundas

In [ ]:
gravity_regional = deep_sources.predict(coordinates)

data = data.assign(gravity_regional_mgal=gravity_regional)
data

We then calculate the **residual** effect by subtracting the regional effect from the Bouguer gravity disturbance

Calculamos el efecto **residual** como el disturbio de Bouguer menos el efecto regional

In [ ]:
gravity_residual = data.gravity_bouguer_mgal - gravity_regional

data = data.assign(gravity_residual_mgal=gravity_residual)
data

Graficamos un mapa del campo residual. Este mapa debe tener semejanzas con lo que conocemos acerca de la geología de Bushveld.

We plot a map of the residual field. This map should bear some resemblance to what we know about the geology of the Bushveld.

In [ ]:
scatter_plot_gravity(
    data.easting_m,
    data.northing_m,
    data.gravity_residual_mgal,
    title="Residual gravity",
)

## Grid the residuals | Grillando los campos residuales

One way to improve the display of our results is to interpolate them into a regular grid at constant altitude.

The best way to do this is by **equivalent sources**. We are going to use the same technique as in the previous step, but now the sources will not be located at great depths, but rather shallower.
Once we fit the coefficients of the sources, we are going to predict their gravitational effects on a regular grid at constant altitude.

Una forma de mejorar la visualización de nuestros resultados es interpolarlos en una grilla regular a altitud constante.

La mejor forma de hacer esto es mediante **fuentes equivalentes**. Vamos a utilizar la misma técnica que en el paso anterior, pero ahora las fuentes no estarán ubicadas a grandes profundidades, sino más someras.
Una vez que ajustemos los coeficientes de las fuentes, vamos a predecir sus efectos gravitatorios en una grilla regular a altitud constante.

We define the equivalent sources located at a depth of 10km and with a damping of 10 (we do not want very smooth solutions) and adjust them with the data

Definimos las fuentes equivalentes ubicadas a 10km de profundidad y con un damping de 10 (no queremos soluciones muy suaves) y las ajustamos con los datos

In [ ]:
eqs = hm.EquivalentSources(damping=10, depth=10e3)

coordinates = (data.easting_m, data.northing_m, data.height_sea_level_m)
eqs.fit(coordinates, data.gravity_residual_mgal)

We calculate the effect of the sources on a regular grid at 2200 meters altitude, with a resolution of 2 arc minutes

Calculamos el efecto de las fuentes sobre una grilla regular a 2200 metros de altitud, con una resolución de 2 minutos de arco.

In [ ]:
residual_grid = eqs.grid(
    upward=2200,  # height in meters
    spacing=3000,  # spacing in meters
    data_names=["gravity_residual"],
)
residual_grid

Graficamos la grilla

Plot the grid

In [ ]:
plt.figure(figsize=(12, 12))
residual_grid.gravity_residual.plot()
plt.title("Residual gravity grid")
plt.gca().set_aspect("equal")
plt.show()

---

## Homework | Tarea

**EN**:

Now it's your turn to process gravity data!

To do that, we are going to use a set of gravity data over Alice Springs, Australia.
You can find the data file in https://raw.githubusercontent.com/santisoler/coco-fatiando-tutorial/main/homework_data/australia_gravity.csv.
The checksum hash of this file is: `sha256:b0f44e5b523a3ca034d07820022c23e5b65875956a88e6eda4cd69f9b66da9fc`

The goal is to write a new notebook that carry out the following tasks:

1. Import the packages needed to download, process and plot the data.
1. Download the gravity data of Alice Springs using Pooch.
1. Download the Earth topography grid in the `earth-topography-10arcmin.nc` (doi: 10.5281/zenodo.5167357) file using Pooch.
1. Load the gravity data using Pandas and the topography grid using Xarray.
1. Crop the topography grid to the data region plus a 5 degrees padding.
    > Help: we can use the `verde.get_region()` function to obtain the data region and then use `verde.pad_region()` to pad it.
1. Project the gravity data and the topography grid using a Mercator projection. Use the mean latitude of the data as the central latitude.
1. Compute the **gravity disturbance** by removing the normal gravity from the data. To do so, compute the **normal gravity** generated by the WGS84 ellipsoid through Boule.
1. Compute the **terrain effect** through Harmonica. You can use the same density values we used in this notebook for the prisms. Then, compute the **Bouguer gravity** as the difference between the **gravity disturbance** and the **terrain effect**.
1. Split the Bouguer gravity in the **regional field** and the **residual field**. You can compute the regional field through deep equivalent sources (`harmonica.EquivalentSources`).
1. **Grid** the residual field onto a regular grid at a 1300m height with a spacing of 9km.

Finally, **create a new** `coco-fatiando-2022` **repo** in your GitHub account and upload the notebook you wrote.

You can plot the data while you perform you computations. Besides, we recommend to add every new result to the DataFrame and wrote Markdown cells to briefly explain what is the code doing.

**ES**:

¡Ahora es tu turno de procesar datos gravimétricos!

Para ello vamos a usar un conjunto de datos gravimétricos sobre Alice Springs, Australia.
Puedes encontrar este archivo en https://raw.githubusercontent.com/santisoler/coco-fatiando-tutorial/main/homework_data/australia_gravity.csv.
La suma de comprobación (hash) de ese archivo es: `sha256:b0f44e5b523a3ca034d07820022c23e5b65875956a88e6eda4cd69f9b66da9fc`

El objetivo es escribir un nuevo notebook en el cual se realicen las siguientes tareas:

1. Importar las librerías necesarias para descargar, procesar y graficar los datos.
1. Descargar los datos gravimétricos de Alice Springs utilizando Pooch.
1. Descargar la grilla de topografía presente en el archivo `earth-topography-10arcmin.nc` (doi: 10.5281/zenodo.5167357) utilizando Pooch.
1. Cargar los datos gravimétricos con Pandas y la grilla de topografía con Xarray.
1. Recortar la grilla de topografía a la región de los datos gravimétricos, incluyendo 5 grados de padding a cada lado.
    > Ayuda: podemos utilizar la función `verde.get_region()` para obtener la región de un conjunto de coordenadas, y luego `verde.pad_region()` para agregar padding a la región.
1. Proyectar los datos gravimétricos y la grilla de topografía utilizando una proyección de tipo Mercator, con la latitud central ubicada en la latitud media de los datos.
1. Calcular el **disturbio de gravedad** quitando la gravedad normal de los datos observados. Para ello, calcula la gravedad normal generada por el elipsoide WGS84 mediante Boule.
1. Calcula el **efecto gravitatorio de la topografía** mediante Harmonica. Utiliza la mismas densidades que elegimos en este notebook. Calcula luego la **gravedad de Bouguer** como la diferencia entre el **disturbio de gravedad** y el **efecto topográfico**.
1. Separa la gravedad de Bouguer en el **campo regional** y el **campo residual**. Puedes calcular el campo regional mediante fuentes equivalentes (`harmonica.EquivalentSources`) profundas.
1. **Grilla** el campo residual en una grilla regular a una altitud de 1300m y con una resolución de 9km.

Finalmente, **crea un repositorio** `coco-fatiando-2022` en tu cuenta de GitHub y sube el notebook que escribiste.

Puedes graficar los datos a medida que vas realizando las tareas. Además, recomendamos agregar cada nuevo cálculo al DataFrame de datos y escribir celdas Markdown en las que detalles brevemente qué realizas en las celdas de código.

---

## License

All Python source code is made available under the BSD 3-clause license. You
can freely use and modify the code, without warranty, so long as you provide
attribution to the authors.

Unless otherwise specified, all figures and Jupyter notebooks are available
under the Creative Commons Attribution 4.0 License (CC-BY).

The full text of these licenses is provided in the [`LICENSE.txt`](LICENSE.txt)
file.